In [6]:
from filterpy.kalman import KalmanFilter
f = KalmanFilter (dim_x=7, dim_z=4)

In [7]:
#!pip install filterpy

In [8]:
f.R

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [20]:
from datetime import datetime
import torch
torch.pi = torch.acos(torch.zeros(1)).item() * 2 # which is 3.1415927410125732
# import torch.nn as nn
from Linear_sysmdl import SystemModel
# from Extended_data import DataGen,DataLoader,DataLoader_GPU, Decimate_and_perturbate_Data,Short_Traj_Split
#from Extended_data import N_E, N_CV, N_T, F, H, T, T_test, m1_0, m2_0, m, n
from Extended_data import N_T, F, H, T, T_test, m1_0, m2_0
from Pipeline_KF import Pipeline_KF
from KalmanNet_nn import KalmanNetNN
import torch
torch.pi = torch.acos(torch.zeros(1)).item() * 2 # which is 3.1415927410125732
import torch.nn as nn
from Linear_sysmdl import SystemModel
from Extended_data import DataGen,DataLoader,DataLoader_GPU, Decimate_and_perturbate_Data,Short_Traj_Split
from Extended_data import N_E, N_CV, N_T, F, H, T, T_test, m1_0, m2_0, m, n
from Pipeline_KF import Pipeline_KF
from KalmanNet_nn import KalmanNetNN
from datetime import datetime

from KalmanFilter_test import KFTest

from Plot import Plot_RTS as Plot

dataFolderName = 'Simulations/Linear_canonical/H=I' + '/'
#dataFileName = ['2x2_rq-1010_T100.pt','2x2_rq020_T100.pt','2x2_rq1030_T100.pt','2x2_rq2040_T100.pt','2x2_rq3050_T100.pt']
# print("Start Data Gen")
dataFileName = ['7x7_rq020_T100.pt']
# DataGen(sys_model, dataFolderName + dataFileName[index], T, T_test,randomInit=False)
print("Data Load")
[train_input, train_target, cv_input, cv_target, test_input, test_target] = DataLoader_GPU(dataFolderName + dataFileName[0])
print("trainset size:",train_target.size())
print("cvset size:",cv_target.size())
print("testset size:",test_target.size())


dataFileName = ['7x7_rq020_T100.pt']
modelFolder = 'KNet' + '/'
today = datetime.today()
now = datetime.now()
strToday = today.strftime("%m.%d.%y")
strNow = now.strftime("%H:%M:%S")
strTime = strToday + "_" + strNow
r2 = torch.tensor([0.1], dtype=torch.float32)
vdB = -20 # ratio v=q2/r2
v = 10**(vdB/10)
q2 = torch.mul(v,r2)

#for index in range(0,len(r2)):

print("1/r2 [dB]: ", 10 * torch.log10(1/r2[0]))
print("1/q2 [dB]: ", 10 * torch.log10(1/q2[0]))

# True model
r = torch.sqrt(r2[0])
q = torch.sqrt(q2[0])
sys_model = SystemModel(F, q, H, r, T, T_test)
sys_model.InitSequence(m1_0, m2_0)

KNet_Pipeline = Pipeline_KF(strTime, "KNet", "KNet_"+ dataFileName[0])
KNet_Pipeline.setssModel(sys_model)
KNet_model = KalmanNetNN()
KNet_model.Build(sys_model)
KNet_Pipeline.setModel(KNet_model)

KNet_Pipeline.model = torch.load(modelFolder+"model_KNet_7x7_rq020_T100.pt", map_location=torch.device('cpu'))
print(KNet_Pipeline.model)
[KNet_MSE_test_linear_arr, KNet_MSE_test_linear_avg, KNet_MSE_test_dB_avg, KNet_test] = KNet_Pipeline.NNTest(N_T, test_input, test_target)


Data Load
trainset size: torch.Size([1000, 7, 100])
cvset size: torch.Size([100, 7, 100])
testset size: torch.Size([200, 7, 100])
1/r2 [dB]:  tensor(10.)
1/q2 [dB]:  tensor(30.)
KalmanNetNN(
  (KG_l1): Linear(in_features=14, out_features=1120, bias=True)
  (KG_relu1): ReLU()
  (rnn_GRU): GRU(1120, 980)
  (KG_l2): Linear(in_features=980, out_features=196, bias=True)
  (KG_relu2): ReLU()
  (KG_l3): Linear(in_features=196, out_features=49, bias=True)
)
KNet_7x7_rq020_T100.pt-MSE Test: tensor(276.1209) [dB]
KNet_7x7_rq020_T100.pt-STD Test: tensor(inf) [dB]
Inference Time: 16.234220266342163
